In [2]:
import requests as re
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from datetime import datetime

In [3]:
pd.set_option('display.max_colwidth', None)

## Get forum, sub-forum and forum url categories
#### ignore first

hwz has multiple forum category / branch

Example
1. Forums
2. Lifestyle
3. Eat-drink-man-woman
4. Chill-out / Hangout Den
5. Multiple thread url eg EDMW Mahjong club
6. Thread posts

In [5]:
site_url = 'https://forums.hardwarezone.com.sg'

Build a table of forums and subforums

### Forum here refers to 3rd or 4th level, where we get 5th level thread links (webpage has multiple thread links)

- 3rd level can have threads & contain additional sub-forums which is 4th level

Example:
https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/page-3?last_days=7

- 3rd level refers to Eat-Drink-Man-Woman
- 4th level refers to Chill-out / Hangout Den, TV-Mania & AnimeMania, Events, Entertainment & Celebrity Buzz

In [6]:
def getThreads(forum_url):
    lastForumPage = False
    forum_cols = ['forum_url', 'thread', 'thread_url']
    forum_df = pd.DataFrame(columns=forum_cols)
    forum_page_url = forum_url
    print('hello')
    while(not lastForumPage):
        r2 = re.get(forum_page_url)
        forum_page = r2.text
        print(r2)
#         print(r2.text)
        forum_page_soup = BeautifulSoup(forum_page, 'html.parser')

        if (forum_page_soup.find('a', class_="pageNav-jump pageNav-jump--next") == None):
            lastForumPage = True
        else:
            forum_page_url = site_url + forum_page_soup.find('a', class_="pageNav-jump pageNav-jump--next")['href']
#         print(forum_page_soup.find_all('div', class_="structItem-title"))
        for element in forum_page_soup.find_all('div', class_="structItem-title"):
#             print(element.a)
            row = pd.DataFrame([[forum_url, element.a.contents[0], site_url + element.a['href']]], columns=forum_cols)
            if(len(forum_df)==0):
                forum_df = row
            else:
                forum_df = forum_df.append(row, ignore_index=True) #df.append doesn't work inplace
    return forum_df

Scraped from these links:
https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion

https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/?last_days=90

In [7]:
%%time
# parenting = getThreads('https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/')
cohd = getThreads('https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion')
print(len(cohd))
cohd

hello
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
95
CPU times: user 625 ms, sys: 28.7 ms, total: 654 ms
Wall time: 1.72 s


,forum_url,thread,thread_url
0,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,No Nut November / NoFap,https://forums.hardwarezone.com.sg/threads/no-nut-november-nofap.6404691/
1,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,Fake accounts that pop up during every saga in SG,https://forums.hardwarezone.com.sg/threads/fake-accounts-that-pop-up-during-every-saga-in-sg.6424934/
2,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,Recommend brand/model blood pressure monitor machine,https://forums.hardwarezone.com.sg/threads/recommend-brand-model-blood-pressure-monitor-machine.6415588/
3,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,[OFFICIAL] BTSS Clubbing Outing @ Zouk 21th July,https://forums.hardwarezone.com.sg/threads/official-btss-clubbing-outing-zouk-21th-july.5869159/
4,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,Local Works....,https://forums.hardwarezone.com.sg/threads/local-works.6441221/
...,...,...,...
90,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,Need help on my issue:(,https://forums.hardwarezone.com.sg/threads/need-help-on-my-issue.6516603/
91,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,Do you think that there are unhappiness in MTF?,https://forums.hardwarezone.com.sg/threads/do-you-think-that-there-are-unhappiness-in-mtf.6604610/
92,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,Do u guys prefer installment or lump sum payment?,https://forums.hardwarezone.com.sg/threads/do-u-guys-prefer-installment-or-lump-sum-payment.6522857/
93,https://forums.hardwarezone.com.sg/forums/chill-out-hangout-den.234/?last_days=365&order=last_post_date&direction=asc&thread_type=discussion,EDMW Mahjong Club,https://forums.hardwarezone.com.sg/threads/edmw-mahjong-club.1664058/


In [12]:
%%time
# parenting = getThreads('https://forums.hardwarezone.com.sg/parenting-kids-early-learning-397/')
parenting = getThreads('https://forums.hardwarezone.com.sg/forums/eat-drink-man-woman.16/?last_days=90')
print(len(parenting))
parenting

hello
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Respons

KeyboardInterrupt: 

In [8]:
cohd.to_csv(r'chill-out-hangout-den.234_lastupdate365_threadlinks.csv', mode='a')

In [27]:
parenting.to_csv(r'Eat-Drink-Man-Woman.16_lastupdate90days_threadlinks.csv', mode='a')

Get threads in a forum/sub-forum - all pages

In [9]:
def getPosts(thread_url):
    #print(thread_url)
    lastThreadPage = False
    thread_cols = ['thread_url', 'userid', 'timestamp', 'post_text', 'post_number', 'post_id', 'first_quote_post_number'] # 'likes_userid'
    thread_df = pd.DataFrame(columns=thread_cols)
    thread_page_url = thread_url

    while(not lastThreadPage):
        #print(thread_page_url)
        r3 = re.get(thread_page_url)
        thread_page = r3.text
        thread_page_soup = BeautifulSoup(thread_page, 'html.parser')

        if (thread_page_soup.find('a', class_="pageNav-jump pageNav-jump--next") == None):
            lastThreadPage = True
        else:
            thread_page_url = site_url + thread_page_soup.find('a', class_="pageNav-jump pageNav-jump--next")['href']

        thread_page_posts = thread_page_soup.find('div', {'class': 'block-body js-replyNewMessageContainer'})
        
        try: 
            for post in thread_page_posts.find_all('article', {'class': 'message message--post js-post js-inlineModContainer'}):
                userid_url = post.find('a', {'class': 'username'})['href']
                userid = post.find('a', {'class': 'username'})['data-user-id']

                iso_datetime = post.find('time', {'itemprop': 'datePublished' })['datetime']

                post_text = ""
                
                first_quote_post_number = np.nan
                
                try:
                    post_text = post.find('div', {'class': 'bbWrapper'}).get_text(' ', strip=True)
                    if(post.find('div', {'class': 'bbWrapper'}).find('blockquote', {'class': 'bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch'})):
                        first_quote = post.find('blockquote', {'class': 'bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch'})
                        first_quote_post_number = first_quote.find('a', {'class': 'bbCodeBlock-sourceJump'})['data-content-selector'].split('#')[1].lstrip('post-')                    
                        post.find('div', {'class': 'bbWrapper'}).blockquote.decompose()
                        post_text = post.find('div', {'class': 'bbWrapper'}).get_text(' ', strip=True)
                except AttributeError as e: 
                    pass
                

                post_number = int(post.find('ul', {'class': 'message-attribution-opposite message-attribution-opposite--list'}).find_all('li')[1].a.text.split('#')[1])

                post_id = post.find('div', {'class': 'message-userContent lbContainer js-lbContainer'})['data-lb-id'].split('post-')[1]

                row = pd.DataFrame([[thread_url, userid, iso_datetime, post_text, post_number, post_id, first_quote_post_number]], columns=thread_cols)
                if(len(thread_df)==0):
                    thread_df = row
                else:
                    thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
        except:
            row = pd.DataFrame([[thread_url, "", "", "", np.nan, np.nan, np.nan]], columns=thread_cols) #posts missing, thread may have been deleted
            if(len(thread_df)==0):
                thread_df = row
            else:
                thread_df = thread_df.append(row, ignore_index=True) #df.append doesn't work inplace
    thread_df['post_text'] = thread_df['post_text'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))

    return thread_df

In [8]:
#19 threads
# parenting['thread_url']

In [10]:
# posts_df = getPosts(parenting['thread_url'][0])
# posts_df.query('first_quote_post_number == first_quote_post_number')
# posts_df

posts_df = pd.DataFrame(columns=['thread_url', 'userid', 'timestamp', 'post_text', 'post_number', 'post_id', 'first_quote_post_number'])
posts_df

,thread_url,userid,timestamp,post_text,post_number,post_id,first_quote_post_number


In [10]:
# parenting['thread_url'][0]

'https://forums.hardwarezone.com.sg/threads/no-nut-november-nofap.6404691/'

In [11]:
for p in parenting['thread_url']:
    pointofchildren = getPosts(p)
    pointofchildren.query('first_quote_post_number == first_quote_post_number')
    posts_df = pd.concat([posts_df, pointofchildren], axis=0)
posts_df

NameError: name 'parenting' is not defined

In [26]:
posts_df.to_csv(r'Eat-Drink-Man-Woman.16_lastupdate90days.csv', mode='a')
# posts_df.to_csv(r'chill-out-hangout-den.234_lastupdate365.csv', header=None, mode='a')